In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import filter_ops

import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import numpy as np
import timeit

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
tf.reset_default_graph()

np.random.seed(0)
probs_shape = [4*4, 8]
probs = np.random.normal(size=probs_shape).astype(np.float32)

features_shape = [4*4, 3]
#features = np.random.randint(0, 255, size=features_shape).astype(np.float32) 
features = np.random.normal(0, 10, size=features_shape).astype(np.float32)

probs = tf.convert_to_tensor(probs)
features = tf.convert_to_tensor(features)

lattice = filter_ops.permuto_init(features)
out_cpp = filter_ops.permuto_compute(probs, *lattice)
out_tf = filter_ops.ph_filter(probs, lattice)

with tf.Session() as sess:
    err = tf.test.compute_gradient_error(probs, probs_shape, out_cpp, probs_shape, 
                                         delta=1e-4)
    print(f'max gradient error (C++): {err}')

    err = tf.test.compute_gradient_error(probs, probs_shape, out_tf, probs_shape, 
                                         delta=1e-4)
    print(f'max gradient error (TF): {err}')
    

max gradient error (C++): 0.0005471110343933105


/home/bagautdi/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


max gradient error (TF): 0.00043904781341552734
